# Scalar test

A single numeric value in and a single numeric value out but as a dimensionless scalar value.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.15 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [1]:
import tensorflow as tf

In [3]:
x = tf.placeholder(tf.float32, shape=(), name="input")
w = tf.Variable(10, dtype=tf.float32, name="w")
b = tf.Variable(5, dtype=tf.float32, name="b")

z = tf.add(tf.multiply(x, w), b, name="output")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    z_eval = sess.run([z], feed_dict={x: 2})
    
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input": x}, outputs={"output": z})
    
    print(z_eval) # 25

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb
[25.0]


## Freeze the graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output
```

## Convert the model to a tensorflow lite format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input \
	--output_arrays=output
```

# Adding Quantization (Unused)

In [ ]:
tf.reset_default_graph()



## Quantize the model in a tflite format

Use the `toco` command line utility to convert the frozen graph to a quantized tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input_x \
	--input_shapes=1 \
	--output_arrays=output_z \
    --inference_type=QUANTIZED_UINT8 \
    --std_dev_values=127 \
    --mean_values=12 
```